In [84]:
import pandas as pd
import numpy as np
import zipfile
from pathlib import Path
import gc
from tqdm import tqdm
import pyarrow.parquet as pq
import pyarrow as pa
import psutil
import shutil

In [2]:
data_dir = Path("/workspace/projects/binance-public-data/python/data")
header = ['open_time',
        'open',
        'high',
        'low',
        'close',
        'volume',
        'close_time',
        'quote_volume',
        'count',
        'taker_buy_volume',
        'taker_buy_quote_volume',
        'ignore'
    ]
TRADE_HEADERS = ["id",
"price",	
#Price
"qty",	
#Quantity
"base_qty",	
#Base Quantity
"time",
#Time in unix time format
"is_buyer_maker",
"is_best_match"
]	


AGG_TRADE_HEADERS = ["agg_trade_id",	
# Aggregated Trade ID
"price",	
# Price
"quantity",	
# Quantity
"first_trade_id",	
# First Trade ID
"last_trade_id",	
# Last Trade ID
"transact_time",	
# Transaction time in unix format
"is_buyer_maker"]

In [81]:
def memory_usage():
    process = psutil.Process()
    r = process.memory_info().rss / (1024 ** 3)
    return r

def read_kline_data(data_dir, header):
    month_dir = Path(data_dir) / f"spot/monthly/klines"
    day_dir = Path(data_dir) / f"spot/daily/klines" 
    
    target_pairs_month = [path.name for path in month_dir.glob("*")]
    target_pairs_day = [path.name for path in day_dir.glob("*")]
    print(target_pairs_day)
    assert set(target_pairs_month) == set(target_pairs_day)
    
    intervals = [interval.name for interval in (month_dir / target_pairs_month[0]).glob("*")]
    
    data_dic = {}
    for pair in target_pairs_month:
        data_dic[pair] = {interval:None for interval in intervals}
    
    for pair in tqdm(data_dic):
        for interval in data_dic[pair]:
            temp_holder = []
            for zip_file in (month_dir / pair / interval).glob("**/*.zip"):
                temp_holder.append(pd.read_csv(zip_file, header=None).values)
            for zip_file in (day_dir / pair / interval).glob("**/*.zip"):
                temp_holder.append(pd.read_csv(zip_file, header =None).values)
            temp = pd.DataFrame(np.concatenate(temp_holder))

            temp.columns = header
            data_dic[pair][interval] = temp
            
        
        #print(memory_usage())
        gc.collect()
    return data_dic

def read_data(data_dir, target, header):
    # processs path
    assert target in {"aggTrades", "trades"}, f"valid targets are aggTrades, klines, trades"
    month_dir = Path(data_dir) / f"spot/monthly/{target}"
    day_dir = Path(data_dir) / f"spot/daily/{target}" 
    
    target_pairs_month = [path.name for path in month_dir.glob("*")]
    target_pairs_day = [path.name for path in day_dir.glob("*")]
    # print(target_pairs_day)
    assert set(target_pairs_month) == set(target_pairs_day)    
    
    
    target_pairs = target_pairs_month
    data_dic = {}
    for pair in target_pairs:
        data_dic[pair] = None

    for pair in tqdm(data_dic):
        temp_holder = []
        for zip_file in (month_dir / pair).glob("**/*.zip"):
            temp_holder.append(pd.read_csv(zip_file, header=None))
            #gc.collect()
            print(memory_usage(), zip_file)
        for zip_file in (day_dir / pair).glob("**/*.zip"):
            temp_holder.append(pd.read_csv(zip_file, header =None))
        temp = pd.concat(temp_holder, ignore_index=True)
        
        #print(count)
        # return temp
    
        temp.columns = header
        data_dic[pair] = temp
        
        # show this epoch memory usage
        # print(memory_usage())
        gc.collect()
    return data_dic

def read_trade_zip_and_store_to_arrow(data_dir, target, pair, header):
    assert target in {"aggTrades", "trades"}, f"valid targets are aggTrades, klines, trades"
    month_dir = Path(data_dir) / f"spot/monthly/{target}"
    day_dir = Path(data_dir) / f"spot/daily/{target}" 
    
    temp_holder = []
    for zip_file in (month_dir / pair).glob("**/*.zip"):
        temp_holder.append(pd.read_csv(zip_file, header=None))
        #gc.collect()
        # print(memory_usage(), zip_file)
    for zip_file in (day_dir / pair).glob("**/*.zip"):
        temp_holder.append(pd.read_csv(zip_file, header =None))
    temp = pd.concat(temp_holder, ignore_index=True)

    temp.columns = header
    
    new_save_dir = Path("/workspace/projects/binance-public-data/python/data/yingruiz_custom/klines")
    df = pa.Table.from_pandas(temp)
    path = new_save_dir / f"{pair}-{target}.parquet"
    pq.write_table(df, path)
    
    print(memory_usage(), shutil.disk_usage("/workspace"))
    
    
    
    
    
    

In [25]:
import psutil
psutil.virtual_memory()

svmem(total=270059012096, available=170595020800, percent=36.8, used=97078005760, free=165688168448, active=15343468544, inactive=82648432640, buffers=19423232, cached=7273414656, shared=30707712, slab=4642025472)

In [27]:
170595020800 / 1024 ** 3

158.8789939880371

In [26]:
270059012096 / 1024 ** 3

251.51205444335938

In [43]:
aggTrades_month_lst = data_dir / f"spot/monthly/{'aggTrades'}/BNBUSDT"
trades_month_lst = data_dir / f"spot/monthly/{'trades'}/BNBUSDT"
a = list(aggTrades_month_lst.glob("**/*.zip"))[42]
c = list(trades_month_lst.glob("**/*.zip"))[42]

In [44]:
b = pd.read_csv(c)

In [51]:
memory_usage()

2.5013656616210938

In [46]:
del b

In [33]:
del a

NameError: name 'a' is not defined

In [55]:
gc.collect()

1842

In [73]:
a = pd.read_csv("/workspace/projects/binance-public-data/python/data/spot/monthly/trades/BNBUSDT/BNBUSDT-trades-2021-02.zip", header = None)
b = pd.read_csv("/workspace/projects/binance-public-data/python/data/spot/monthly/trades/BNBUSDT/BNBUSDT-trades-2021-03.zip", header = None)

In [75]:
c = pd.read_csv("/workspace/projects/binance-public-data/python/data/spot/monthly/trades/BNBUSDT/BNBUSDT-trades-2020-03.zip", header = None)

In [78]:
print(a.shape)
new = pd.concat([a,b,c], ignore_index=True)

(45727149, 7)


In [69]:
memory_usage()

11.804935455322266

In [68]:
del a

In [60]:
a.memory_usage()

NameError: name 'a' is not defined

In [3]:
data = read_kline_data(data_dir, header=header)

NameError: name 'data_dir' is not defined

In [83]:
gc.collect()

882

In [82]:
trade_data = read_data(data_dir, "trades", header = TRADE_HEADERS)

  0%|          | 0/28 [00:00<?, ?it/s]

8.642505645751953 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/SUNUSDT/SUNUSDT-trades-2020-09.zip
8.642505645751953 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/SUNUSDT/SUNUSDT-trades-2020-10.zip
8.642505645751953 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/SUNUSDT/SUNUSDT-trades-2020-11.zip
8.642505645751953 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/SUNUSDT/SUNUSDT-trades-2020-12.zip
8.642505645751953 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/SUNUSDT/SUNUSDT-trades-2021-01.zip
8.642505645751953 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/SUNUSDT/SUNUSDT-trades-2021-02.zip
8.642505645751953 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/SUNUSDT/SUNUSDT-trades-2021-03.zip
8.642505645751953 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/SUNUSDT/SUNUSDT-trades-2021-04.zip


  4%|▎         | 1/28 [00:19<08:33, 19.02s/it]

10.076522827148438 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/BNBUSDT/BNBUSDT-trades-2017-11.zip
10.076522827148438 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/BNBUSDT/BNBUSDT-trades-2017-12.zip
10.076522827148438 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/BNBUSDT/BNBUSDT-trades-2018-01.zip
10.076522827148438 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/BNBUSDT/BNBUSDT-trades-2018-02.zip
10.076522827148438 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/BNBUSDT/BNBUSDT-trades-2018-03.zip
10.076522827148438 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/BNBUSDT/BNBUSDT-trades-2018-04.zip
10.076522827148438 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/BNBUSDT/BNBUSDT-trades-2018-05.zip
10.076522827148438 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/BNBUSDT/BNBUSDT-trades-2018

  7%|▋         | 2/28 [07:00<1:45:50, 244.26s/it]

44.550113677978516 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/RAREUSDT/RAREUSDT-trades-2021-10.zip
44.550113677978516 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/RAREUSDT/RAREUSDT-trades-2021-11.zip
44.550113677978516 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/RAREUSDT/RAREUSDT-trades-2021-12.zip
44.550113677978516 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/RAREUSDT/RAREUSDT-trades-2022-01.zip
44.550113677978516 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/RAREUSDT/RAREUSDT-trades-2022-02.zip
44.550113677978516 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/RAREUSDT/RAREUSDT-trades-2022-03.zip
44.550113677978516 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/RAREUSDT/RAREUSDT-trades-2022-04.zip
44.550113677978516 /workspace/projects/binance-public-data/python/data/spot/monthly/trades/RAREUSDT/RARE

  7%|▋         | 2/28 [07:15<1:34:19, 217.66s/it]


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [18]:
trade_data = read_data(data_dir, "trades", header = TRADE_HEADERS)

  0%|          | 0/28 [00:00<?, ?it/s]

0.7282257080078125
0.7411651611328125
0.7550811767578125
0.7758331298828125
0.8158721923828125
0.9335479736328125
1.3092002868652344
1.7023696899414062
1.8342056274414062
2.199108123779297
2.457164764404297
2.877758026123047
3.297943115234375
3.8250808715820312
4.182415008544922
4.460224151611328
4.718029022216797
4.905040740966797
5.161388397216797
5.294933319091797
5.457530975341797
5.542980194091797
5.590343475341797
5.610851287841797
5.625988006591797
5.871299743652344
5.907676696777344
5.927452087402344
5.945762634277344
5.978233337402344
6.006553649902344
6.022911071777344
6.040489196777344
6.051963806152344
6.067100524902344
6.076622009277344
6.083213806152344
6.097129821777344
6.121299743652344
6.198204040527344
6.243370056152344
6.369346618652344
6.537403106689453
6.589160919189453
6.632862091064453
6.671436309814453
6.714893341064453
7.847446441650391


  4%|▎         | 1/28 [00:27<12:35, 27.96s/it]

8.682598114013672
8.784893035888672
9.152812957763672
9.618389129638672
10.119205474853516
10.594684600830078
10.909381866455078
11.279010772705078
11.495807647705078
11.635456085205078
11.740680694580078
11.811725616455078
11.889118194580078
12.038532257080078
12.214313507080078
12.562145233154297
13.076072692871094
13.551021575927734
14.527626037597656
15.21683120727539
15.853996276855469
16.244224548339844
16.57294464111328
16.94664764404297
17.284088134765625
17.559768676757812
17.95340347290039
18.683650970458984
19.588314056396484
20.334186553955078
20.898639678955078
21.291370391845703
22.154449462890625
23.327205657958984
25.371742248535156
26.235492706298828
27.60406494140625
29.379127502441406
32.11770248413086
42.733280181884766
49.26029586791992
62.739707946777344
73.78775787353516
80.79904174804688
84.59790420532227
90.8784065246582
94.72976303100586
99.5665512084961
106.13178253173828
110.11508560180664
113.44094848632812
115.74219512939453
117.85037994384766
119.76470184

: 

#### save data to paquet file

In [9]:
lst_float

[8.682598114013672,
 8.784893035888672,
 9.152812957763672,
 9.618389129638672,
 10.119205474853516,
 10.594684600830078,
 10.909381866455078,
 11.279010772705078,
 11.495807647705078,
 11.635456085205078,
 11.740680694580078,
 11.811725616455078,
 11.889118194580078,
 12.038532257080078,
 12.214313507080078,
 12.562145233154297,
 13.076072692871094,
 13.551021575927734,
 14.527626037597656,
 15.21683120727539,
 15.853996276855469,
 16.244224548339844,
 16.57294464111328,
 16.94664764404297,
 17.284088134765625,
 17.559768676757812,
 17.95340347290039,
 18.683650970458984,
 19.588314056396484,
 20.334186553955078,
 20.898639678955078,
 21.291370391845703,
 22.154449462890625,
 23.327205657958984,
 25.371742248535156,
 26.235492706298828,
 27.60406494140625,
 29.379127502441406,
 32.11770248413086,
 42.733280181884766,
 49.26029586791992,
 62.739707946777344,
 73.78775787353516,
 80.79904174804688,
 84.59790420532227,
 90.8784065246582,
 94.72976303100586,
 99.5665512084961,
 106.131782

In [13]:
import numpy as np
import pandas as pd
lst_float = [float(i) for i in lst.splitlines()]
tmp = pd.Series(lst_float).diff() 
tmp.idxmax()

41

In [1]:
lst = """8.682598114013672
8.784893035888672
9.152812957763672
9.618389129638672
10.119205474853516
10.594684600830078
10.909381866455078
11.279010772705078
11.495807647705078
11.635456085205078
11.740680694580078
11.811725616455078
11.889118194580078
12.038532257080078
12.214313507080078
12.562145233154297
13.076072692871094
13.551021575927734
14.527626037597656
15.21683120727539
15.853996276855469
16.244224548339844
16.57294464111328
16.94664764404297
17.284088134765625
17.559768676757812
17.95340347290039
18.683650970458984
19.588314056396484
20.334186553955078
20.898639678955078
21.291370391845703
22.154449462890625
23.327205657958984
25.371742248535156
26.235492706298828
27.60406494140625
29.379127502441406
32.11770248413086
42.733280181884766
49.26029586791992
62.739707946777344
73.78775787353516
80.79904174804688
84.59790420532227
90.8784065246582
94.72976303100586
99.5665512084961
106.13178253173828
110.11508560180664
113.44094848632812
115.74219512939453
117.85037994384766
119.76470184326172
122.64155960083008
124.13260650634766
125.71688842773438
127.39742279052734
128.605712890625
129.73766326904297
132.43456649780273
133.62428283691406
135.15800857543945
136.6778564453125
138.59083557128906
140.2331771850586
141.44458389282227
143.0424461364746
144.2618179321289
145.15122985839844
145.78619003295898
146.7367706298828
148.56216430664062
150.8032341003418
152.62826538085938
154.28997039794922
"""

In [ ]:
# save each kline pair-interval to a pache file
new_save_dir = Path("/workspace/projects/binance-public-data/python/data/yingruiz_custom/klines")
for pair in data:
    for interval in intervals:
        df = pa.Table.from_pandas(data[pair][interval])
        path = new_save_dir / f"{pair}-{interval}-klines.parquet"
        pq.write_table(df, path)

In [ ]:
# save each trade pair to a pache file
new_save_dir = Path("/workspace/projects/binance-public-data/python/data/yingruiz_custom/klines")
for pair in trade_data:
    df = pa.Table.from_pandas(trade_data[pair])
    path = new_save_dir / f"{pair}-trades.parquet"
    pq.write_table(df, path)

In [ ]:
# save each AggTrade pair to a pache file
new_save_dir = Path("/workspace/projects/binance-public-data/python/data/yingruiz_custom/klines")
for pair in agg_trade_data:
    df = pa.Table.from_pandas(agg_trade_data[pair])
    path = new_save_dir / f"{pair}-agg_trades.parquet"
    pq.write_table(df, path)

In [ ]:
agg_trade_data = read_data(data_dir, "trades", header = TRADE_HEADERS)

In [9]:
intervals = list(data["SUNUSDT"].keys())

In [ ]:
def read_kline_and_save_to_arrow_file(data_dir, header):
    month_dir = Path(data_dir) / f"spot/monthly/klines"
    day_dir = Path(data_dir) / f"spot/daily/klines" 
    
    

In [11]:
month_dir = Path(data_dir) / f"spot/monthly/klines"
day_dir = Path(data_dir) / f"spot/daily/klines" 

In [23]:
tmp = list(month_dir.glob("**/*.zip"))[0]


In [24]:
tmp

PosixPath('/workspace/projects/binance-public-data/python/data/spot/monthly/klines/SUNUSDT/1m/SUNUSDT-1m-2020-09.zip')

In [25]:
tmp.stem = "www"

AttributeError: can't set attribute

In [28]:
for path in list(month_dir.glob("**/*.zip")):
    df = pd.read_csv(path, header = None)
    df.columns = header
    df_table = pa.Table.from_pandas(df)
    new_path = path.parent / f"{path.stem}.parquet"
    pq.write_table(df_table, new_path)

In [29]:
for path in list(day_dir.glob("**/*.zip")):
    df = pd.read_csv(path, header = None)
    df.columns = header
    df_table = pa.Table.from_pandas(df)
    new_path = path.parent / f"{path.stem}.parquet"
    pq.write_table(df_table, new_path)

In [45]:
def read_kline_data_from_pq(data_dir, header):
    month_dir = Path(data_dir) / f"spot/monthly/klines"
    day_dir = Path(data_dir) / f"spot/daily/klines" 
    
    target_pairs_month = [path.name for path in month_dir.glob("*")]
    target_pairs_day = [path.name for path in day_dir.glob("*")]
    #print(target_pairs_day)
    assert set(target_pairs_month) == set(target_pairs_day)
    
    intervals = [interval.name for interval in (month_dir / target_pairs_month[0]).glob("*")]
    
    data_dic = {}
    for pair in target_pairs_month:
        data_dic[pair] = {interval:None for interval in intervals}
    
    for pair in tqdm(data_dic):
        for interval in data_dic[pair]:
            temp_holder = []
            for path in (month_dir / pair / interval).glob("**/*.parquet"):
                temp_holder.append(pq.read_table(path).to_pandas().values)
                #temp_holder.append(pd.read_csv(zip_file, header=None).values)
            for path in (day_dir / pair / interval).glob("**/*.parquet"):
                temp_holder.append(pq.read_table(path).to_pandas().values)
                #temp_holder.append(pd.read_csv(zip_file, header =None).values)
            temp = pd.DataFrame(np.concatenate(temp_holder))

            temp.columns = header
            data_dic[pair][interval] = temp
            
        
        #print(memory_usage())
        gc.collect()
    return data_dic

In [46]:
r = read_kline_data_from_pq(data_dir = data_dir, header = header)

  4%|▎         | 1/28 [00:01<00:51,  1.90s/it]


ArrowInvalid: Schema at index 2 was different: 
open_time: int64
open: double
high: double
low: double
close: double
volume: double
close_time: int64
quote_volume: double
count: int64
taker_buy_volume: double
taker_buy_quote_volume: double
ignore: double
vs
open_time: int64
open: double
high: double
low: double
close: double
volume: double
close_time: int64
quote_volume: double
count: int64
taker_buy_volume: double
taker_buy_quote_volume: double
ignore: int64

In [37]:
np.sum(np.sum(r["BNBUSDT"]["15m"] == data["BNBUSDT"]["15m"])) == data["BNBUSDT"]["15m"].shape[0] * data["BNBUSDT"]["15m"].shape[1]

/root/miniconda3/envs/crypto_trading/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


True

In [38]:
df_1 = pd.DataFrame({'year': [2020, 2022, 2019, 2021],
                   'n_legs': [2, 4, 5, 100],
                   'animals': ["Flamingo", "Horse", "Brittle stars", "Centipede"]})
tb_1 = pa.Table.from_pandas(df_1)

In [39]:
df_2 = pd.DataFrame({'year': [2020, 2022, 2019, 2021],
                   'n_legs': [2, 4, 5, 100],
                   'animals': ["Flamingo", "Horse", "Brittle stars", "Centipede"]})
tb_2 = pa.Table.from_pandas(df_2)

In [43]:
tmp_111 = pa.concat_tables([tb_1, tb_2],unify_schemas=False)

In [44]:
tmp_111.to_pandas()

,year,n_legs,animals
0,2020,2,Flamingo
1,2022,4,Horse
2,2019,5,Brittle stars
3,2021,100,Centipede
4,2020,2,Flamingo
5,2022,4,Horse
6,2019,5,Brittle stars
7,2021,100,Centipede


In [48]:
# save each pair-interval to a pache file
new_save_dir = Path("/workspace/projects/binance-public-data/python/data/yingruiz_custom/klines")
for pair in data:
    for interval in intervals:
        df = pa.Table.from_pandas(data[pair][interval])
        path = new_save_dir / f"{pair}-{interval}-klines.parquet"
        pq.write_table(df, path)

In [7]:
new_data = {}
new_save_dir = Path("/workspace/projects/binance-public-data/python/data/yingruiz_custom/klines")
for path in new_save_dir.glob("**/*.parquet"):
    pair, interval, _ = path.stem.split("-")
    if not pair in new_data:
        new_data[pair] = {}
    new_data[pair][interval] = pd.read_parquet(path, engine='pyarrow')


In [55]:
np.sum(data["SUNUSDT"]["15m"], axis=(0,1))

ValueError: No axis named (0, 1) for object type DataFrame

In [10]:
for pair in data:
    for interval in intervals:
        shape_1 = new_data[pair][interval].shape
        shape_2 = data[pair][interval].shape
        assert shape_1 == shape_2
        assert np.sum(np.sum(data[pair][interval] == new_data[pair][interval] )) == shape_2[0] * shape_2[-1]
        

/root/miniconda3/envs/crypto_trading/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


In [11]:
data[pair][interval].sum()

open_time                 3.421219e+14
open                      8.829290e+00
high                      1.007749e+01
low                       7.683260e+00
close                     8.786320e+00
volume                    2.207577e+11
close_time                3.422465e+14
quote_volume              1.173636e+10
count                     3.178302e+07
taker_buy_volume          1.122966e+11
taker_buy_quote_volume    5.995476e+09
ignore                    0.000000e+00
dtype: float64

In [12]:
pd.__version__

'2.2.2'